In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize

from keras.models import Sequential, Model
from keras.layers import Dense, Input, GRU, Dropout
from keras.callbacks import ModelCheckpoint
from keras.saving import load_model
from keras.src.layers import concatenate

from sklearn.preprocessing import MinMaxScaler

from scripts.process_dataset import read_dataset, read_simulation
from utils import draw_prediction, plot_loss

2025-04-28 11:00:29.181013: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def predict(X, models, scaler):
    """
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(X)
    """
    X_trans = scaler.transform(X)
    Y_pred_trans = np.concatenate([model.predict(X_trans, verbose=0) for model in models], axis=1)
    Y_pred_trans_extended = np.concatenate([Y_pred_trans, np.zeros((Y_pred_trans.shape[0], X.shape[1]-Y_pred_trans.shape[1]))], axis=1)
    Y_pred = scaler.inverse_transform(Y_pred_trans_extended)[:, :Y_pred_trans.shape[1]]
    return Y_pred

In [5]:
def next_state(state, next_sterile_pop, model):
    return np.concatenate([model.predict(state, verbose=0)] + [next_sterile_pop], axis=1).reshape(1, -1)

In [7]:
def moving_predict(X, model, scaler):
    X_trans = scaler.transform(X)
    initial_state = X_trans[0:1, :]
    sterile_pop = X_trans[:, 70:]
    states = [initial_state]
    last_state = initial_state
    for i in range(X.shape[0]-1):
        print(i)
        last_state = next_state(last_state, sterile_pop[i+1:i+2, :], model)
        states.append(last_state)
    Y_pred_trans = np.concatenate(states, axis=0)[:70]
    Y_pred_trans_extended = np.concatenate([Y_pred_trans, np.zeros((Y_pred_trans.shape[0], X.shape[1]-Y_pred_trans.shape[1]))], axis=1)
    Y_pred = scaler.inverse_transform(Y_pred_trans_extended)[:, :Y_pred_trans.shape[1]]
    return Y_pred

In [6]:
DATASET_NAME = "B"
N_NEURAL = 32
DATASET_PATH = f"../dataset/{DATASET_NAME}"
X, Y = read_dataset(f"{DATASET_PATH}/train")
X = X.reshape((X.shape[0], X.shape[2]))
Y_extended = np.c_[Y, np.zeros((Y.shape[0], X.shape[1]-Y.shape[1]))]
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X)
X_trans = scaler.transform(X)
Y_trans = scaler.transform(Y_extended)[:, :Y.shape[1]]

In [5]:
Y.shape

(9520, 70)

In [10]:
model = get_model(1000, X_trans, Y_trans)

In [ ]:
#plot_model(model)
X_test, Y_test = read_simulation(f"{DATASET_PATH}/test/1")
Y_pred = scaler.inverse_transform(np.concatenate([model.predict(scaler.transform(X_test), verbose=0), np.zeros((X_test.shape[0], X_test.shape[1]-Y_test.shape[1]))], axis=1))
for i in range(10):
    draw_prediction(Y_test, Y_pred, i)


In [ ]:
plot_model(get_model(N_NEURAL, X_trans, Y_trans, i, f"../models/{DATASET_NAME}"))

In [30]:
X_test, Y_test = read_simulation(f"{DATASET_PATH}/test/1")
X_test = X_test[:30, :]
Y_test = Y_test[:30, :]
Y_pred = moving_predict(X_test, model, scaler)
for i in range(10):
    draw_prediction(Y_test, Y_pred, i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


NameError: name 'draw_prediction' is not defined